In [1]:
import cProfile

from cfg import Cfg

from source import SourceCfg, Dimension, SpatialDistribution, AngularDistribution
from source import Source

from detector import DetectorCfg, Measurement

from sample import Sample, Layer, Material

from direct_problem import DirectProblemCfg
from direct_problem import DirectProblem


from support_functions import timeit, KW

import numpy as np

In [2]:
source_cfg = SourceCfg(dimension=Dimension.surface,
                       spatial_distribution=SpatialDistribution.gauss,
                       angular_distribution=AngularDistribution.collimated,
                       beam_center=np.array([0, 0, 0]),
                       beam_diameter=float(1))

detector_cfg = DetectorCfg(measurement=Measurement.CollimatedDiffuse)

direct_problem_cfg = DirectProblemCfg(N=10000,
                                      threads=1)

cfg = Cfg(
    source=source_cfg,
    detector=detector_cfg,
    direct_problem=direct_problem_cfg
    )

cfg.validate()

sample = Sample([
    Layer(material=Material.transparent, depth=1.,
          mu_a=0.0001, mu_s=0., g=0., n=1.45),
    Layer(material=Material.scattering, depth=1.,
          mu_a=1., mu_s=1., g=0.9, n=1.35),
    Layer(material=Material.transparent, depth=1.,
          mu_a=0.0001, mu_s=0., g=0.0, n=1.45)
          ])

detector = cfg.detector.get_detector()
source = Source(cfg.source, sample)

direct_problem = DirectProblem(cfg.direct_problem,
                               sample, source, detector)

result = timeit(direct_problem.solve)()
print(result)
print(np.sum(result[0]) + result[1])
print(KW)

(array([1252.01119047, 1821.49939932,   51.1782515 ,  347.43849106]), 6527.872667651335)
10000.000000000226
{'SOLVE': {'calls': 1, 'times': 6624}}


In [3]:
"""
Профилирование
ncalls – это количество совершенных вызовов;
tottime – это все время, потраченное в данной функции;
percall – ссылается на коэффициент tottime, деленный на ncalls;
cumtime – совокупное время, потраченное как в данной функции, так и наследуемых функциях. Это работает также и с рекурсивными функциями!
Второй столбец percall – это коэффициент cumtime деленный на примитивные вызовы;
filename:lineno(function) предоставляет соответствующие данные о каждой функции
"""
cProfile.run("direct_problem.solve()", sort = 'tottime')

         2008245 function calls (1930202 primitive calls) in 6.809 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    25325    1.184    0.000    2.965    0.000 direct_problem.py:95(move)
    45051    1.099    0.000    1.487    0.000 direct_problem.py:325(reflection)
     5054    0.557    0.000    0.721    0.000 ffi.py:149(__call__)
    15325    0.552    0.000    0.590    0.000 direct_problem.py:180(turn)
    10000    0.464    0.000    6.521    0.001 direct_problem.py:48(trace)
    20000    0.225    0.000    0.225    0.000 ufunclike.py:199(isneginf)
    43134    0.142    0.000    0.142    0.000 direct_problem.py:378(R_frenel)
    74287    0.103    0.000    0.103    0.000 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
    24995    0.100    0.000    0.100    0.000 {method 'random' of 'numpy.random.mtrand.RandomState' objects}
44430/7932    0.099    0.000    0.141    0.000 ir.py:313(_rec_list_vars)
   296542    0